---
title: "Model Selection & Evaluation: Making the Right Choices"
author: "Ram Polisetti"
date: "2024-03-19"
categories: [machine-learning, model-selection, evaluation, tutorial]
image: "model_selection.jpg"
description: "A comprehensive guide to selecting and evaluating machine learning models, with practical examples and best practices."
jupyter: python3
---




# Model Selection & Evaluation in Machine Learning

Choosing the right model and properly evaluating its performance are crucial skills in machine learning. This post provides a comprehensive guide to these essential topics.

## Setup and Data Loading


In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.datasets import load_diabetes, load_breast_cancer
from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from sklearn.metrics import mean_squared_error, r2_score, confusion_matrix
from sklearn.linear_model import LogisticRegression, Ridge, Lasso
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.svm import SVC

# Set style for visualizations
plt.style.use('seaborn')
sns.set_theme(style="whitegrid")

# Load datasets
X_cancer, y_cancer = load_breast_cancer(return_X_y=True)
X_diabetes, y_diabetes = load_diabetes(return_X_y=True)

# Split datasets
X_cancer_train, X_cancer_test, y_cancer_train, y_cancer_test = train_test_split(
    X_cancer, y_cancer, test_size=0.2, random_state=42
)

X_diabetes_train, X_diabetes_test, y_diabetes_train, y_diabetes_test = train_test_split(
    X_diabetes, y_diabetes, test_size=0.2, random_state=42
)

# Scale features
scaler = StandardScaler()
X_cancer_train_scaled = scaler.fit_transform(X_cancer_train)
X_cancer_test_scaled = scaler.transform(X_cancer_test)
X_diabetes_train_scaled = scaler.fit_transform(X_diabetes_train)
X_diabetes_test_scaled = scaler.transform(X_diabetes_test)

## Understanding Different Models

### 1. Classification Models Comparison


In [ ]:
# Initialize classifiers
classifiers = {
    'Logistic Regression': LogisticRegression(random_state=42),
    'Decision Tree': DecisionTreeClassifier(random_state=42),
    'Random Forest': RandomForestClassifier(random_state=42),
    'SVM': SVC(random_state=42),
    'Gradient Boosting': GradientBoostingClassifier(random_state=42)
}

# Train and evaluate each classifier
results = []
for name, clf in classifiers.items():
    # Train the model
    clf.fit(X_cancer_train_scaled, y_cancer_train)
    
    # Make predictions
    y_pred = clf.predict(X_cancer_test_scaled)
    
    # Calculate metrics
    results.append({
        'Model': name,
        'Accuracy': accuracy_score(y_cancer_test, y_pred),
        'Precision': precision_score(y_cancer_test, y_pred),
        'Recall': recall_score(y_cancer_test, y_pred),
        'F1 Score': f1_score(y_cancer_test, y_pred)
    })

# Create results DataFrame
results_df = pd.DataFrame(results)
print("\nClassification Models Comparison:")
print(results_df.round(3))

# Visualize results
plt.figure(figsize=(12, 6))
results_melted = pd.melt(results_df, id_vars=['Model'], 
                        value_vars=['Accuracy', 'Precision', 'Recall', 'F1 Score'])
sns.barplot(data=results_melted, x='Model', y='value', hue='variable')
plt.title('Classification Models Performance Comparison')
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()

### 2. Cross-Validation Strategies


In [ ]:
from sklearn.model_selection import KFold, StratifiedKFold, TimeSeriesSplit

def compare_cv_strategies(X, y, model, cv_strategies):
    results = []
    
    for name, cv in cv_strategies.items():
        scores = cross_val_score(model, X, y, cv=cv, scoring='accuracy')
        results.append({
            'Strategy': name,
            'Mean Score': scores.mean(),
            'Std Score': scores.std()
        })
    
    return pd.DataFrame(results)

# Define cross-validation strategies
cv_strategies = {
    'KFold (k=5)': KFold(n_splits=5, shuffle=True, random_state=42),
    'StratifiedKFold': StratifiedKFold(n_splits=5, shuffle=True, random_state=42),
    'TimeSeriesSplit': TimeSeriesSplit(n_splits=5)
}

# Compare strategies using Random Forest
rf_classifier = RandomForestClassifier(random_state=42)
cv_results = compare_cv_strategies(X_cancer_scaled, y_cancer, rf_classifier, cv_strategies)

print("\nCross-Validation Strategies Comparison:")
print(cv_results.round(3))

# Visualize CV results
plt.figure(figsize=(10, 6))
sns.barplot(data=cv_results, x='Strategy', y='Mean Score', 
            yerr=cv_results['Std Score'], capsize=0.1)
plt.title('Cross-Validation Strategies Comparison')
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()

### 3. Hyperparameter Tuning


In [ ]:
# Define parameter grid for Random Forest
param_grid = {
    'n_estimators': [100, 200, 300],
    'max_depth': [None, 10, 20],
    'min_samples_split': [2, 5, 10]
}

# Initialize GridSearchCV
grid_search = GridSearchCV(
    RandomForestClassifier(random_state=42),
    param_grid,
    cv=5,
    scoring='accuracy',
    n_jobs=-1
)

# Fit grid search
grid_search.fit(X_cancer_train_scaled, y_cancer_train)

# Print best parameters and score
print("\nGrid Search Results:")
print("Best parameters:", grid_search.best_params_)
print("Best cross-validation score:", round(grid_search.best_score_, 3))

# Create DataFrame of CV results
cv_results = pd.DataFrame(grid_search.cv_results_)
cv_results = cv_results[['params', 'mean_test_score', 'std_test_score']]
cv_results = cv_results.sort_values('mean_test_score', ascending=False).head()

print("\nTop 5 Parameter Combinations:")
print(cv_results.round(3))

### 4. Learning Curves Analysis


In [ ]:
from sklearn.model_selection import learning_curve

def plot_learning_curves(estimator, X, y, title):
    train_sizes, train_scores, test_scores = learning_curve(
        estimator, X, y, cv=5, n_jobs=-1,
        train_sizes=np.linspace(0.1, 1.0, 10),
        scoring='accuracy'
    )
    
    train_mean = np.mean(train_scores, axis=1)
    train_std = np.std(train_scores, axis=1)
    test_mean = np.mean(test_scores, axis=1)
    test_std = np.std(test_scores, axis=1)
    
    plt.figure(figsize=(10, 6))
    plt.plot(train_sizes, train_mean, label='Training score')
    plt.plot(train_sizes, test_mean, label='Cross-validation score')
    
    plt.fill_between(train_sizes, train_mean - train_std, train_mean + train_std, alpha=0.1)
    plt.fill_between(train_sizes, test_mean - test_std, test_mean + test_std, alpha=0.1)
    
    plt.title(title)
    plt.xlabel('Training Examples')
    plt.ylabel('Score')
    plt.legend(loc='best')
    plt.grid(True)
    plt.tight_layout()

# Plot learning curves for different models
models = {
    'Random Forest': RandomForestClassifier(random_state=42),
    'SVM': SVC(random_state=42),
    'Logistic Regression': LogisticRegression(random_state=42)
}

for name, model in models.items():
    plot_learning_curves(model, X_cancer_scaled, y_cancer, f'Learning Curves - {name}')
    plt.show()

### 5. Model Calibration


In [ ]:
from sklearn.calibration import calibration_curve, CalibratedClassifierCV

def plot_calibration_curves(clf, X_train, X_test, y_train, y_test, name):
    # Train and calibrate classifier
    clf.fit(X_train, y_train)
    calibrated_clf = CalibratedClassifierCV(clf, cv=5)
    calibrated_clf.fit(X_train, y_train)
    
    # Get predictions
    prob_pos = clf.predict_proba(X_test)[:, 1]
    prob_pos_calibrated = calibrated_clf.predict_proba(X_test)[:, 1]
    
    # Calculate calibration curves
    fraction_of_positives, mean_predicted_value = calibration_curve(
        y_test, prob_pos, n_bins=10)
    fraction_of_positives_cal, mean_predicted_value_cal = calibration_curve(
        y_test, prob_pos_calibrated, n_bins=10)
    
    # Plot calibration curves
    plt.figure(figsize=(10, 6))
    plt.plot([0, 1], [0, 1], 'k--', label='Perfectly calibrated')
    plt.plot(mean_predicted_value, fraction_of_positives, 
             label=f'Original {name}')
    plt.plot(mean_predicted_value_cal, fraction_of_positives_cal, 
             label=f'Calibrated {name}')
    
    plt.xlabel('Mean predicted probability')
    plt.ylabel('Fraction of positives')
    plt.title(f'Calibration Curves - {name}')
    plt.legend()
    plt.grid(True)
    plt.tight_layout()

# Plot calibration curves for different models
for name, model in models.items():
    plot_calibration_curves(model, X_cancer_train_scaled, X_cancer_test_scaled,
                          y_cancer_train, y_cancer_test, name)
    plt.show()

## Best Practices for Model Selection

1. **Start Simple**
   - Begin with simple models (e.g., logistic regression)
   - Establish baseline performance
   - Gradually increase complexity

2. **Consider Model Characteristics**
   - Training time and computational resources
   - Model interpretability
   - Prediction speed
   - Memory usage

3. **Validation Strategy**
   - Use appropriate cross-validation
   - Consider data distribution
   - Account for time series if applicable

4. **Hyperparameter Tuning**
   - Start with broad search
   - Refine promising areas
   - Use random or grid search
   - Consider Bayesian optimization

## Common Pitfalls to Avoid

1. **Overfitting**
   - Using too complex models
   - Not enough regularization
   - Not using cross-validation

2. **Data Leakage**
   - Information leak in preprocessing
   - Wrong cross-validation strategy
   - Test set contamination

3. **Wrong Metrics**
   - Using inappropriate metrics
   - Not considering business context
   - Ignoring model calibration

4. **Poor Validation**
   - Not using cross-validation
   - Wrong splitting strategy
   - Ignoring data distribution

## Model Selection Framework

1. **Problem Analysis**
   - Define objectives
   - Understand constraints
   - Identify success metrics

2. **Data Assessment**
   - Size and quality
   - Feature characteristics
   - Target distribution

3. **Model Shortlisting**
   - Consider problem type
   - Resource constraints
   - Interpretability needs

4. **Evaluation Strategy**
   - Choose metrics
   - Select validation approach
   - Define baseline

5. **Implementation**
   - Start simple
   - Iterate and improve
   - Document findings

## Conclusion

Model selection and evaluation are critical steps in the machine learning pipeline. Key takeaways:

1. Start with simple models and establish baselines
2. Use appropriate validation strategies
3. Consider multiple evaluation metrics
4. Be aware of common pitfalls
5. Document your process and findings

In the next post, we'll explore deep learning fundamentals and neural network architectures.

## Additional Resources

1. Books:
   - "Hands-On Machine Learning" by Aurélien Géron
   - "Applied Predictive Modeling" by Max Kuhn

2. Online Resources:
   - Scikit-learn Model Selection Guide
   - Machine Learning Mastery Model Selection Tutorials
   - Papers with Code Benchmarks

Remember: The best model is often not the most complex one, but the one that best balances performance, interpretability, and practical constraints.